In [12]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate

template = PromptTemplate.from_template(
    "Hi, what would be the best copy for a {business} website?"
)
prompt = template.format(business="golf lounge")

chat = ChatOpenAI(temperature=0.1)

chat.predict(prompt)

"Welcome to the Ultimate Golf Lounge Experience!\n\nAt [Golf Lounge Name], we are passionate about creating a haven for golf enthusiasts like you. Whether you're a seasoned golfer or just starting out, our state-of-the-art facility is designed to provide you with an unparalleled golfing experience.\n\nStep into our luxurious lounge and be greeted by a warm and inviting atmosphere. Our modern and stylish decor sets the perfect backdrop for relaxation and enjoyment. With comfortable seating areas, a fully stocked bar, and high-definition TVs broadcasting live golf tournaments, you'll feel right at home.\n\nBut what truly sets us apart is our cutting-edge golf simulators. Immerse yourself in a virtual world where you can play on world-renowned courses, challenge your friends to a friendly competition, or even receive professional coaching from our experienced instructors. Our simulators offer realistic graphics, precise ball tracking, and accurate swing analysis, ensuring that every swing

In [14]:
from langchain.schema import SystemMessage, HumanMessage, AIMessage

template = ChatPromptTemplate.from_messages([
    ("system", "You are a copywriter for digital products. You only reply in {language}."),
    ("ai", "Hi, I am your chatbot."),
    ("human", "Hi, what would be the best copy for a {business} website?")])


prompt = template.format_messages(
    language="Spanish",
    business="Coffe shop"
)

chat.predict_messages(prompt)

AIMessage(content='¡Bienvenidos al mejor lugar para disfrutar del café! En nuestra cafetería, encontrarás el equilibrio perfecto entre sabor, calidad y ambiente acogedor. Nuestros expertos baristas están listos para deleitarte con una amplia variedad de cafés de origen, cuidadosamente seleccionados y preparados con pasión. Desde un aromático espresso hasta un suave cappuccino, cada taza es una experiencia única que despertará tus sentidos. Además, ofrecemos deliciosos postres y bocadillos que complementarán tu experiencia cafetera. Ven y descubre por qué somos el destino favorito de los amantes del café. ¡Te esperamos con una taza caliente y una sonrisa!')